<a href="https://colab.research.google.com/github/arcadyb/pyreport/blob/main/pdffromjsonheb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install Pillow reportlab
from reportlab.pdfbase.pdfmetrics import stringWidth

In [9]:
!pip install Pillow reportlab
!pip install python-bidi arabic-reshaper
from bidi.algorithm import get_display
import arabic_reshaper  # While mainly for Arabic, it's needed for bidi processing

import json
from google.colab import drive
from PIL import Image
from reportlab.pdfgen import canvas
import os
import tempfile
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
# Mount Google Drive
drive.mount('/content/drive')

# Install necessary packages
fontfile='NotoSansHebrew-VariableFont_wdthwght.ttf'
font_path = '/content/drive/My Drive/ColabWSData/reports/NotoSansHebrew-VariableFont_wdthwght.ttf'

# Register the font with ReportLab
pdfmetrics.registerFont(TTFont('NotoSansHebrew', font_path))
# Function to rotate images
def rotate_image(img, angle):
    return img.rotate(angle, expand=True)

# Define the folder path and the JSON file path
folder_path = '/content/drive/My Drive/ColabWSData/reports/makolet1'
json_file_path = '/content/drive/My Drive/ColabWSData/reports/text.txt'
output_pdf = '/content/drive/My Drive/ColabWSData/reports/kabalot2_with_toc.pdf'

# Load the JSON data
with open(json_file_path, 'r', encoding='utf-8') as file:
    image_files_info = json.load(file)

# Initialize PDF canvas
c = canvas.Canvas(output_pdf)

# Adjusted height for larger text and TOC
title_height = 140

# Process and add images to the PDF
for info in image_files_info:
    image_path = os.path.join(folder_path, info['filename'])
    with Image.open(image_path) as img:
        # Assuming rotation is already handled
        rotated_img = rotate_image(img, int(info['rot']))

        # Get the original image dimensions
        width, height = rotated_img.size

        # Define the extra space needed at the top for the title and its margin
        title_space = 80  # Adjust based on your font size and desired margin

        # Calculate the new page height to include the image, title, and margin
        page_height = height + title_space

        c.setPageSize((width, page_height))

        # Temporarily save the rotated image for adding to the PDF
        with tempfile.NamedTemporaryFile(delete=True, suffix='.png') as tmp:
            rotated_img.save(tmp.name, format='PNG')
            # Position the image at the bottom of the extra space reserved for the title
            c.drawImage(tmp.name, 0, 0, width=width, height=height)  # Image is drawn starting from the bottom

        # Process the Hebrew text with bidi algorithm
        hebrew_text = info['title']
        reshaped_text = arabic_reshaper.reshape(hebrew_text)  # Prepare text for bidi processing
        bidi_text = get_display(reshaped_text)  # Apply bidi algorithm

        # Draw the title text above the image
        c.setFont("NotoSansHebrew", 48)
        # Draw the title at the top, considering the margin
        text_y_position = height + (title_space - 50)  # Adjust the subtraction to move text down
        c.drawCentredString(width / 2, text_y_position, bidi_text)
        c.showPage()
c.save()
print("PDF created successfully.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
PDF created successfully.
